| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information:  https://github.com/edgi-govdata-archiving/ECHO-Cross-Program
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue.
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Nationwide statistics about environmental enforcement and compliance trends

## Setup
Here we load some helper code to get us going.

In [ ]:
# Import code libraries
!pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules &>/dev/null;
!pip install geopandas &>/dev/null;

import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import csv
import datetime
import folium
from folium.plugins import FastMarkerCluster
import ipywidgets as widgets
from IPython.core.display import display, HTML
from pandas.errors import EmptyDataError
from ECHO_modules.get_data import get_echo_data

def formatter(value):
  return "{:0.2f}".format(value)

print("Done!")

Done!


Here we set up some code to help us store and eventually export the metrics.

In [ ]:
inspections = dict()
violations = dict()
enforcements = dict()
penalties = dict()
emissions = dict()

## Start getting data
First, get summary data from the ECHO_EXPORTER table.

In [ ]:
from ECHO_modules.get_data import get_echo_data

# Get everything we will need from ECHO_EXPORTER in a single DB query.
# We can then use the full dataframe to specialize views of it.
full_echo_data = None
column_mapping = {
    '"REGISTRY_ID"': str,
    '"FAC_NAME"': str,
    '"FAC_LAT"': float,
    '"FAC_LONG"': float,
    '"AIR_IDS"': str,
    '"NPDES_IDS"': str,
    '"RCRA_IDS"': str,
    '"DFR_URL"': str,
    '"AIR_FLAG"': str,
    '"NPDES_FLAG"': str,
    '"GHG_FLAG"': str,
    '"RCRA_FLAG"': str,
    '"FAC_ACTIVE_FLAG"': str
}
column_names = list( column_mapping.keys() )
columns_string = ','.join( column_names )
sql = 'select ' + columns_string + ' from "ECHO_EXPORTER" where "AIR_FLAG" = \'Y\' or "NPDES_FLAG" = \'Y\' or "GHG_FLAG" = \'Y\' or "RCRA_FLAG" = \'Y\''
try:
    # Don't index.
    full_echo_data = get_echo_data( sql )
except EmptyDataError:
    print("\nThere are no EPA facilities for this query.\n")
full_echo_data

,REGISTRY_ID,FAC_NAME,FAC_LAT,FAC_LONG,AIR_IDS,NPDES_IDS,RCRA_IDS,DFR_URL,AIR_FLAG,NPDES_FLAG,GHG_FLAG,RCRA_FLAG,FAC_ACTIVE_FLAG
0,1.100640e+11,SUMMIT MACHINING LLC,34.033700,-117.587360,NaN,NaN,CAL000330050,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,Y
1,1.100023e+11,EASTEX FOREST PRODUCTS INCORPORATED,29.885080,-95.309310,NaN,TXR05DT28,TXD026631630,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,Y,Y
2,1.100036e+11,"PROSPERITY DRIVE DATA CENTER, LLC",39.059490,-76.964990,MD0000002403102308,NaN,MDR000023143,http://echo.epa.gov/detailed-facility-report?f...,Y,N,N,Y,Y
3,1.100315e+11,FARRELL COOPER MINI/HEAVENER EAST 2 SHAD,34.888380,-94.499110,OK0000004077700081,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,Y,N,N,N,Y
4,1.100646e+11,GRANT-JOHNSON FUNERAL HOMES INC,41.371659,-89.475840,IL000011085ADG,NaN,NaN,http://echo.epa.gov/detailed-facility-report?f...,Y,N,N,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224121,1.100253e+11,"MORGAN TRUCK BODY, LLC",28.011370,-81.872050,FL0000001210500369,FLR05H361,FLR000179762,http://echo.epa.gov/detailed-facility-report?f...,Y,Y,N,Y,Y
2224122,1.100005e+11,GASSER OLDS CO INC,33.996340,-118.224180,NaN,CAZ188674,CAD981672025,http://echo.epa.gov/detailed-facility-report?f...,N,Y,N,Y,Y
2224123,1.100036e+11,FREEPORT AGGREGATE,42.184230,-83.230700,NaN,NaN,MID049263783,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,Y
2224124,1.100704e+11,ADVANCED AUTO REFINISHING INC,33.483343,-117.674611,NaN,NaN,CAL000432087,http://echo.epa.gov/detailed-facility-report?f...,N,N,N,Y,Y


## Number of Currently Regulated Facilities Per Program

In [ ]:
air_fac = full_echo_data.loc[(full_echo_data["AIR_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
water_fac = full_echo_data.loc[(full_echo_data["NPDES_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
waste_fac = full_echo_data.loc[(full_echo_data["RCRA_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]
ghg_fac = full_echo_data.loc[(full_echo_data["GHG_FLAG"]=="Y") & (full_echo_data["FAC_ACTIVE_FLAG"]=="Y")].shape[0]

display(HTML("<h3>There are "+ str(air_fac) + " facilities currently regulated under the Clean Air Act.</h3>"))
display(HTML("<h3>There are "+ str(water_fac) + " facilities currently regulated under the Clean Water Act.</h3>"))
display(HTML("<h3>There are "+ str(waste_fac) + " facilities currently regulated under RCRA (hazardous waste).</h3>"))
display(HTML("<h3>There are "+ str(ghg_fac) + " facilities currently reporting greenhouse gas emissions.</h3>"))

## Clean Air Act inspections in 2022

In [ ]:
# Use SQL to search for and select the data about air stack tests
air_inspections = None
try:
    sql = 'select * from \"ICIS-AIR_FCES_PCES\" where \"ACTUAL_END_DATE\" like \'__-__-2022\''

    # Download the data from that URL
    air_inspections = get_echo_data( sql, 'pgm_sys_id' )
except EmptyDataError:
    print( "No data found")

air_inspections

,PGM_SYS_ID,ACTIVITY_ID,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES
0,0400000013121D0001,3603090713,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,03-11-2022,CAANESH
1,0400000013195D0001,3603091806,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,03-17-2022,CAANESH
2,0400013015EPA01,3603379851,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,06-10-2022,CAAMACT
3,0400013051EPA01,3603379786,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,06-14-2022,CAAMACT
4,0400013051EPA02,3603379808,E,INS,Inspection/Evaluation,PFF,PCE Off-Site,06-15-2022,CAAMACT
...,...,...,...,...,...,...,...,...,...
47055,WV00003100002,3603092788,S,INS,Inspection/Evaluation,FOO,FCE On-Site,03-21-2022,"CAAMACT, CAANSPS, CAAPSD, CAASIP"
47056,WV00002900008,3603362546,S,INS,Inspection/Evaluation,PCE,PCE On-Site,09-20-2022,"CAANESH, CAANSPS, CAASIP"
47057,WV00002900008,3603366397,S,INS,Inspection/Evaluation,FOO,FCE On-Site,09-28-2022,"CAANESH, CAANSPS, CAASIP"
47058,WV00003100004,3603169065,S,INS,Inspection/Evaluation,PCE,PCE On-Site,05-04-2022,CAASIP


In [ ]:
# Number of inspections in 2022 per 1000 regulated facilities
air_inspections_metric = formatter((air_inspections.shape[0] / air_fac) * 1000) # Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["CAA"] = air_inspections_metric
display(HTML("<h3>"+ air_inspections_metric +" inspections per 1000 facilities</h3>"))

## Violations of the Clean Air Act in 2022



In [ ]:
air_violations = None
try:
    sql = 'select * from "ICIS-AIR_VIOLATION_HISTORY" where "EARLIEST_FRV_DETERM_DATE" like \'__-__-2022\' or "HPV_DAYZERO_DATE" like \'__-__-2022\''

    air_violations = get_echo_data( sql, "pgm_sys_id" )

    # Optional: remove "FACIL" violations, which are paperwork violations according to: https://19january2017snapshot.epa.gov/sites/production/files/2013-10/documents/frvmemo.pdf
    # air_violations = air_violations.loc[(air_violations["POLLUTANT_DESCS"]!="FACIL")]
except EmptyDataError:
    print( "No data found")
air_violations

,PGM_SYS_ID,ACTIVITY_ID,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,COMP_DETERMINATION_UID,ENF_RESPONSE_POLICY_CODE,PROGRAM_CODES,PROGRAM_DESCS,POLLUTANT_CODES,POLLUTANT_DESCS,EARLIEST_FRV_DETERM_DATE,HPV_DAYZERO_DATE,HPV_RESOLVED_DATE
0,AR0000000513900037,3603406777,State,AR,NaN,AR000A96699,HPV,CAAMACT CAATVP,MACT Standards (40 CFR Part 63) Title V Permits,300000329,FACIL,05-23-2022,05-23-2022,NaN
1,AR0000000513900037,3603615164,State,AR,NaN,AR000A100834,HPV,CAATVP,Title V Permits,300000329,FACIL,10-18-2022,10-18-2022,NaN
2,IN0000001801900008,3603262724,State,IN,NaN,IN000A93714,HPV,CAAMACT,MACT Standards (40 CFR Part 63),300000675,Magnesium,05-18-2022,08-16-2022,NaN
3,IN0000001817300007,3603458007,State,IN,NaN,IN000A97776,HPV,CAAMACT,MACT Standards (40 CFR Part 63),300000004,Fluoride,05-30-2022,05-30-2022,NaN
4,IN0000001812700085,3602918623,State,IN,NaN,IN000A89692,HPV,CAATVP,Title V Permits,300000018,Hydrochloric acid,10-27-2021,01-25-2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4416,PA000841890,3603573713,State,PA,NaN,PA000A0000F00000003409676,FRV,CAASIP,State Implementation Plan for National Primary...,300000329,FACIL,08-17-2022,NaN,NaN
4417,PA000841890,3603573714,State,PA,NaN,PA000A0000F00000003443729,FRV,CAASIP,State Implementation Plan for National Primary...,300000329,FACIL,10-20-2022,NaN,NaN
4418,MI00000000000P1198,3603646293,State,MI,NaN,MI000AP1198CF0000076560,FRV,CAANSR CAASIP,New Source Review Permit Requirements State Im...,300000329,FACIL,01-07-2022,NaN,NaN
4419,CASJV00006029S1137,3603635030,Local,CA,SJV,CASJVA2000000000000004753,HPV,CAATVP,Title V Permits,300000329,FACIL,06-27-2022,06-27-2022,NaN


In [ ]:
# Number of high priority and federally reportable violations per 1000 regulated facilities
air_violations_metric = formatter((air_violations.shape[0] / air_fac) * 1000) # Divide by regulated facilities and multiply by desired rate (per 1000)
violations["CAA"] = air_violations_metric
display(HTML("<h3>"+air_violations_metric+" violations per 1000 facilities </h3>"))

## Formal Enforcement Actions and Penalties under the Clean Air Act in 2022

In [ ]:
air_enforcements = None
try:
    sql = 'select * from "ICIS-AIR_FORMAL_ACTIONS" where "SETTLEMENT_ENTERED_DATE" like \'__/__/2022\''

    air_enforcements = get_echo_data( sql, "pgm_sys_id" )
except EmptyDataError:
    print( "No data found")
air_enforcements

,PGM_SYS_ID,ACTIVITY_ID,ENF_IDENTIFIER,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,STATE_EPA_FLAG,ENF_TYPE_CODE,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,PENALTY_AMOUNT
0,OH0000000215020233,3600391239,05-2015-5085,JDC,Judicial,E,CIV,Civil Judicial Action,04/01/2022,288000.0
1,NY0000008269900126,3603441833,NY000A0000826990012600472,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,10/11/2022,200000.0
2,AR0000000513100274,3603068497,AR000A200209352,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,04/01/2022,1920.0
3,AR0000000577700342,3603068504,AR000A200209353,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,04/01/2022,1470.0
4,LA0000002210900059,3603019677,LA000A3204194,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,02/17/2022,0.0
...,...,...,...,...,...,...,...,...,...,...
2952,LA0000002205900028,3603222443,LA000A2803383,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,05/19/2022,0.0
2953,LA0000002205900048,3603222454,LA000A2804737,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,05/19/2022,0.0
2954,LA0000002205900051,3603222460,LA000A2803509,AFR,Administrative - Formal,S,SCAAAO,Administrative Order,05/19/2022,0.0
2955,ALJEF0000107300001,3603222779,ALJEFA200211564,AFR,Administrative - Formal,L,SCAAAO,Administrative Order,05/11/2022,52000.0


In [ ]:
# Number of formal actions in 2022 per violation
air_enforcements_metric = formatter(air_enforcements.shape[0]/air_violations.shape[0]) # Formal actions divided by number of violations
enforcements["CAA"] = air_enforcements_metric
display(HTML("<h3>"+air_enforcements_metric +" formal enforcement actions per violation</h3>"))

In [ ]:
# Penalties each year per violating facility
air_penalties = air_enforcements.loc[air_enforcements["PENALTY_AMOUNT"]>0]
air_penalties_metric = formatter(sum(air_penalties["PENALTY_AMOUNT"]) / len(air_violations["PGM_SYS_ID"].unique())) #Divide the sum of penalties by number of violating facilities
air_penalties_max = formatter(max(air_penalties["PENALTY_AMOUNT"]))
air_penalties_min = formatter(min(air_penalties["PENALTY_AMOUNT"]))
penalties["CAA"] = air_penalties_metric
display(HTML("<h3>$"+air_penalties_metric +" per facility in violation</h3>"))
display(HTML("<h3>Max: $"+air_penalties_max +"</h3>"))
display(HTML("<h3>Min: $"+air_penalties_min +"</h3>"))

---

## Clean Water Act inspections in 2022

In [ ]:
# Find facilities with pollutant exceedences
water_inspections = None
try:
    sql = 'select "NPDES_ID", "REGISTRY_ID", "ACTUAL_END_DATE", "STATE_EPA_FLAG"' + \
        ' from "NPDES_INSPECTIONS" where "ACTUAL_END_DATE" like \'__/__/2022\''

    water_inspections = get_echo_data( sql, "NPDES_ID" )
except EmptyDataError:
    print( "No data found")
water_inspections

,REGISTRY_ID,ACTUAL_END_DATE,STATE_EPA_FLAG
NPDES_ID,,,
TX0129267,110055045570,08/02/2022,S
IN0020052,110010831984,01/05/2022,S
MSP091466,110002214354,02/03/2022,S
TXR1599GV,110071149576,03/21/2022,S
TXR1524DW,110070875158,08/17/2022,S
...,...,...,...
PAG123898,110010129057,01/13/2022,S
MS0031828,110002307655,02/09/2022,S
OH0021423,110006263809,01/19/2022,S


In [ ]:
# Number of inspections in 2022 per 1000 regulated facilities
water_inspections_metric = formatter((water_inspections.shape[0] / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["CWA"] = water_inspections_metric
display(HTML("<h3>"+water_inspections_metric +" inspections per 1000 facilities</h3>"))

## Violations of the Clean Water Act in 2022

In [ ]:
# Find facilities with water permit violations
water_violations = None
try:
    sql = 'select * from "NPDES_QNCR_HISTORY" where "YEARQTR" = 20221 or "YEARQTR" = 20222 or "YEARQTR" = 20223 or "YEARQTR" = 20224'
    water_violations = get_echo_data( sql, "NPDES_ID" )
except EmptyDataError:
    print( "No data found")
water_violations

,YEARQTR,HLRNC,NUME90Q,NUMCVDT,NUMSVCD,NUMPSCH
NPDES_ID,,,,,,
AK0000841,20221,N,1,0,0,7
AK0000841,20222,E,4,0,0,7
AK0000841,20223,N,0,0,0,7
AK0000841,20224,N,0,0,0,7
AK0001058,20221,,0,0,0,3
...,...,...,...,...,...,...
WVR107508,20223,U,0,0,0,0
WVR107509,20221,U,0,0,0,0
WVR107509,20222,U,0,0,0,0


In [ ]:
# Number of violations each year per 1000 regulated facilities
# Sum violations
water_violations["Sum"] = water_violations["NUME90Q"]	+ water_violations["NUMCVDT"] + water_violations["NUMSVCD"]	+ water_violations["NUMPSCH"]
water_violations_metric = formatter((np.sum(water_violations["Sum"]) / water_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["CWA"] = water_violations_metric
display(HTML("<h3>"+water_violations_metric+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under the Clean Water Act in 2022

In [ ]:
# Find facilities with pollutant exceedences
water_enforcements = None
try:
    sql = 'select "NPDES_ID", "AGENCY", "ENF_TYPE_DESC", "SETTLEMENT_ENTERED_DATE", "FED_PENALTY_ASSESSED_AMT", "STATE_LOCAL_PENALTY_AMT"' + \
        ' from "NPDES_FORMAL_ENFORCEMENT_ACTIONS" where "SETTLEMENT_ENTERED_DATE" like \'__/__/2022\''

    water_enforcements = get_echo_data( sql, "NPDES_ID" )
except EmptyDataError:
    print( "No data found")
water_enforcements

,AGENCY,ENF_TYPE_DESC,SETTLEMENT_ENTERED_DATE,FED_PENALTY_ASSESSED_AMT,STATE_LOCAL_PENALTY_AMT
NPDES_ID,,,,,
NJG151033,EPA,CWA 309G2A AO For Class I Penalties,03/11/2022,11000.0,NaN
WV0023159,State,State CWA Penalty AO,01/05/2022,NaN,15000.0
KYR004211,State,State Administrative Order of Consent,01/04/2022,NaN,25000.0
IN0039349,State,State CWA Penalty AO,01/20/2022,NaN,3060.0
TX0091243,State,State CWA Penalty AO,01/27/2022,NaN,38812.0
...,...,...,...,...,...
CA0063177,State,State CWA Penalty AO,11/01/2022,NaN,15000.0
IDU100023,State,Civil Judicial Action,06/21/2022,NaN,436800.0
WV0027481,State,State CWA Non Penalty AO,10/08/2022,NaN,NaN


In [ ]:
# Number of formal actions in 2022 per violation
water_enforcements_metric = formatter(water_enforcements.shape[0]/water_violations.shape[0]) # Formal actions divided by number of violations
enforcements["CWA"] = water_enforcements_metric
display(HTML("<h3>"+water_enforcements_metric +" formal enforcement actions per violation</h3>"))

In [ ]:
# Penalties each year per violating facility
# Find violating facilities (not all in NPDES QNCR are violating...)
water_violators = water_violations.loc[water_violations["Sum"]>0]
water_violators = len(water_violators.index.unique())
water_enforcements["StateLocalFedFines"] = water_enforcements["FED_PENALTY_ASSESSED_AMT"].fillna(0) + water_enforcements["STATE_LOCAL_PENALTY_AMT"].fillna(0)
water_penalties = water_enforcements.loc[water_enforcements["StateLocalFedFines"]>0]
water_penalties_metric = formatter(sum(water_penalties["StateLocalFedFines"]) / water_violators) #Divide the sum of penalties by number of penalized facilities
water_penalties_max = formatter(max(water_penalties["StateLocalFedFines"]))
water_penalties_min = formatter(min(water_penalties["StateLocalFedFines"]))
penalties["CWA"] = water_penalties_metric
display(HTML("<h3>$"+water_penalties_metric +" per facility in violation</h3>"))
display(HTML("<h3>Max: $"+water_penalties_max +"</h3>"))
display(HTML("<h3>Min: $"+water_penalties_min +"</h3>"))

## RCRA inspections in 2022

In [ ]:
# Find facilities with pollutant exceedences
waste_inspections = None
try:
    sql = 'select * from "RCRA_EVALUATIONS" where "EVALUATION_START_DATE" like \'__/__/2022\''

    waste_inspections = get_echo_data( sql, "ID_NUMBER" )
except EmptyDataError:
    print( "No data found")
waste_inspections

,ACTIVITY_LOCATION,EVALUATION_IDENTIFIER,EVALUATION_TYPE,EVALUATION_DESC,EVALUATION_AGENCY,EVALUATION_START_DATE,FOUND_VIOLATION
ID_NUMBER,,,,,,,
WVR000536136,WV,001,CEI,COMPLIANCE EVALUATION INSPECTION,S,06/14/2022,N
ALD983167925,AL,001,CEI,COMPLIANCE EVALUATION INSPECTION,S,08/03/2022,Y
ALR000064642,AL,001,CEI,COMPLIANCE EVALUATION INSPECTION,S,08/03/2022,Y
VTD001075894,VT,001,FRR,FINANCIAL RECORD REVIEW,S,04/28/2022,N
KYR000016360,KY,001,CEI,COMPLIANCE EVALUATION INSPECTION,S,07/29/2022,N
...,...,...,...,...,...,...,...
KYR000065847,KY,001,CEI,COMPLIANCE EVALUATION INSPECTION,S,07/19/2022,N
LAR000091165,LA,246,FCI,FOCUSED COMPLIANCE INSPECTION,S,08/16/2022,N
TXR000059816,TX,001,NRR,NON-FINANCIAL RECORD REVIEW,E,01/21/2022,Y


In [ ]:
# Number of inspections in 2022 per 1000 regulated facilities
waste_inspections_metric = formatter((waste_inspections.shape[0] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
inspections["RCRA"] = waste_inspections_metric
display(HTML("<h3>"+waste_inspections_metric+" inspections per 1000 facilities</h3>"))

## Violations of RCRA in 2022

In [ ]:
# Find facilities with pollutant exceedences
waste_violations = None
try:
    sql = 'select * from "RCRA_VIOLATIONS" where "DATE_VIOLATION_DETERMINED" like \'__/__/2022\''

    waste_violations = get_echo_data( sql, "ID_NUMBER" )
except EmptyDataError:
    print( "No data found")
waste_violations

,ACTIVITY_LOCATION,VIOLATION_TYPE,VIOLATION_TYPE_DESC,VIOL_DETERMINED_BY_AGENCY,DATE_VIOLATION_DETERMINED,ACTUAL_RTC_DATE,SCHEDULED_COMPLIANCE_DATE
ID_NUMBER,,,,,,,
MTR000208199,MT,262.A,Standards Applicable to Generators of HW: General,S,11/16/2022,11/22/2022,NaN
TND982090151,TN,273.B,Standards for Universal Waste Management: Stan...,S,02/08/2022,03/21/2022,NaN
TNR000047969,TN,270.A,EPA Administered Permit Programs: the HW Permi...,S,04/21/2022,05/25/2022,NaN
WIR000102855,WI,262.B,Standards Applicable to Generators of HW: Mani...,S,02/09/2022,04/19/2022,04/15/2022
NCD986194306,FL,XXS,State Statutory or Regulatory requirements tha...,S,03/23/2022,11/11/2022,02/17/2016
...,...,...,...,...,...,...,...
NYN008030850,NY,279.C,Standards for Used Oil: Generators,S,04/28/2022,06/18/2022,NaN
TNR000047977,TN,279.C,Standards for Used Oil: Generators,S,02/25/2022,06/14/2022,NaN
PAD014081814,PA,262.A,Standards Applicable to Generators of HW: General,S,06/15/2022,10/07/2022,NaN


In [ ]:
# Number of violations in 2022 per 1000 regulated facilities
waste_violations_metric = formatter((waste_violations.shape[0] / waste_fac) * 1000) #Divide by regulated facilities and multiply by desired rate (per 1000)
violations["RCRA"] = waste_violations_metric
display(HTML("<h3>"+waste_violations_metric+" violations per 1000 facilities</h3>"))

## Enforcement Actions and Penalties under RCRA in 2022

In [ ]:
# Find facilities with enforcement actions
waste_enforcements = None
try:
    sql = 'select * from "RCRA_ENFORCEMENTS" where "ENFORCEMENT_ACTION_DATE" like \'__/__/2022\''

    waste_enforcements = get_echo_data( sql, "ID_NUMBER" )
except EmptyDataError:
    print( "No data found")
waste_enforcements

,ACTIVITY_LOCATION,ENFORCEMENT_IDENTIFIER,ENFORCEMENT_TYPE,ENFORCEMENT_DESC,ENFORCEMENT_AGENCY,ENFORCEMENT_ACTION_DATE,PMP_AMOUNT,FMP_AMOUNT,FSC_AMOUNT,SCR_AMOUNT
ID_NUMBER,,,,,,,,,,
NYD002103216,NY,001,HQ140,LETTER OF INTENT TO INITIATE ENFORCEMENT ACTION,S,02/11/2022,NaN,NaN,NaN,NaN
NJR000030627,NJ,001,HQ120,WRITTEN INFORMAL,S,11/22/2022,NaN,NaN,NaN,NaN
OHD004229845,OH,004,HQ120,WRITTEN INFORMAL,S,12/22/2022,NaN,NaN,NaN,NaN
VAVSQG022240,VA,001,VA119,INSPECTOR FACT FINDING LETTER - Warning letter,S,11/10/2022,NaN,NaN,NaN,NaN
NYR000138727,NY,001,HQ310,FINAL 3008(A) COMPLIANCE ORDER,S,12/21/2022,NaN,38997.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
COD004864245,CO,B3,CO175,COMPLIANCE ADVISORY,S,09/21/2022,NaN,NaN,NaN,NaN
MDR000522320,MD,001,MD251,FIELD CITATION - MD NOV,S,10/07/2022,7810.0,NaN,NaN,NaN
NJD986620813,NJ,001,HQ120,WRITTEN INFORMAL,S,01/13/2022,NaN,NaN,NaN,NaN


In [ ]:
# Number of enforcement actions each year per violation
waste_enforcements_metric = formatter(waste_enforcements.shape[0] / waste_violations.shape[0])
enforcements["RCRA"] = waste_enforcements_metric
display(HTML("<h3>"+waste_enforcements_metric+" enforcement actions per violation</h3>"))

In [ ]:
# Penalties each year per violating facility
waste_penalties = waste_enforcements.loc[waste_enforcements["FMP_AMOUNT"]>0]
waste_penalties_metric = formatter(sum(waste_penalties["FMP_AMOUNT"]) / len(waste_violations.index.unique())) #Divide by penalized facilities
waste_penalties_max = formatter(max(waste_penalties["FMP_AMOUNT"]))
waste_penalties_min = formatter(min(waste_penalties["FMP_AMOUNT"]))
penalties["RCRA"] = waste_penalties_metric
display(HTML("<h3>$"+waste_penalties_metric +" per facility in violation</h3>"))
display(HTML("<h3>Max: $"+waste_penalties_max +"</h3>"))
display(HTML("<h3>Min: $"+waste_penalties_min +"</h3>"))

## Greenhouse Gas Emissions in 2021 (latest data available)

In [ ]:
# Find GHG emissions
ghg_emissions = None
try:
    sql = 'select * from "POLL_RPT_COMBINED_EMISSIONS" where "REPORTING_YEAR" = \'2021\' and "PGM_SYS_ACRNM" = \'E-GGRT\''

    ghg_emissions = get_echo_data( sql)
except EmptyDataError:
    print( "No data found")
ghg_emissions

,REPORTING_YEAR,REGISTRY_ID,PGM_SYS_ACRNM,PGM_SYS_ID,POLLUTANT_NAME,ANNUAL_EMISSION,UNIT_OF_MEASURE,NEI_TYPE,NEI_HAP_VOC_FLAG
0,2021,110066943605,E-GGRT,1011834,Nitrous oxide,1110.944,MTCO2e,NaN,NaN
1,2021,110071159847,E-GGRT,1010583,Methane,237.250,MTCO2e,NaN,NaN
2,2021,110071159847,E-GGRT,1010583,Carbon dioxide,28560.200,MTCO2e,NaN,NaN
3,2021,110000309648,E-GGRT,1004343,Carbon dioxide,47822.200,MTCO2e,NaN,NaN
4,2021,110017310391,E-GGRT,1007385,Methane,136.500,MTCO2e,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21471,2021,110001060574,E-GGRT,1000315,Nitrous oxide,20.562,MTCO2e,NaN,NaN
21472,2021,110001060574,E-GGRT,1000315,Carbon dioxide,36502.000,MTCO2e,NaN,NaN
21473,2021,110037942616,E-GGRT,1004022,Nitrous oxide,29.800,MTCO2e,NaN,NaN
21474,2021,110037942616,E-GGRT,1004022,Carbon dioxide,53071.500,MTCO2e,NaN,NaN


In [ ]:
# Emissions in 2021 per facility
ghg_emissions_metric = formatter(np.nansum(ghg_emissions["ANNUAL_EMISSION"]) / len(ghg_emissions["REGISTRY_ID"].unique())) #Divide by reporting facility
ghg_emissions_fac = ghg_emissions.groupby("PGM_SYS_ID")[["ANNUAL_EMISSION"]].sum() # Group by facility
ghg_emissions_max = formatter(np.nanmax(ghg_emissions_fac["ANNUAL_EMISSION"]))
ghg_emissions_min = formatter(np.nanmin(ghg_emissions_fac.loc[ghg_emissions_fac["ANNUAL_EMISSION"]>0]["ANNUAL_EMISSION"]))
emissions["GHG"] = ghg_emissions_metric
display(HTML("<h3>"+ghg_emissions_metric+" MTCO2e (metric tons of carbon dioxide equivalent) emissions per reporting facility</h3>"))
display(HTML("<h3>Max: "+ghg_emissions_max+" MTCO2e (metric tons of carbon dioxide equivalent) emissions</h3>"))
display(HTML("<h3>Min: "+ghg_emissions_min+" MTCO2e (metric tons of carbon dioxide equivalent) emissions</h3>"))
ghg_emissions_fac

,ANNUAL_EMISSION
PGM_SYS_ID,
1000001,637259.640
1000002,116830.596
1000003,79993.496
1000005,78060.732
1000007,15259.592
...,...
1014433,155694.774
1014434,54381.600
1014438,35726.396


# Data Export

In [ ]:
data = [inspections,
violations,
enforcements,
penalties,
emissions]

units = ["#inspections per 1000",
"#violations per 1000",
"#actions per facility in violation",
"$ per facility in violation",
"amount of emissions (metric tons)"]

short_units = ["inspectionsper1000",
"violationsper1000",
"enforcementsperviolatingfacility",
"penaltiesperviolatingfacility",
"emissions2021"]

for index, program in enumerate(data):
    # create dataframe
    df = pd.DataFrame(program, index=[0]).T
    df = df.rename(columns={0: units[index]})
    filename= short_units[index]+"_All_USA_pg4_2022.csv"
    df.to_csv(filename)
    print(df)

     #inspections per 1000
CAA                 241.16
CWA                 124.65
RCRA                 39.30
     #violations per 1000
CAA                 22.66
CWA               1026.33
RCRA                29.28
     #actions per facility in violation
CAA                                0.67
CWA                                0.01
RCRA                               0.40
     $ per facility in violation
CAA                     87011.52
CWA                      1070.01
RCRA                     2480.19
    amount of emissions (metric tons)
GHG                         384211.53
